In [1]:
import tensorflow as tf
import numpy as np
import os
import dataloader
import architecture
import metrics

In [5]:
ROOT_DIR = '.'
DATA_DIR = 'BRATS_2018'
LABELED_DIR = 'MICCAI_BraTS_2018_Data_Training'
UNLABELED_DIR = 'MICCAI_BraTS_2018_Data_Validation'
LABELED_PATH = os.path.join(ROOT_DIR, DATA_DIR, LABELED_DIR)
UNLABELED_PATH = os.path.join(ROOT_DIR, DATA_DIR, UNLABELED_DIR)

BATCH_SIZE = 32
BRAIN_SLICES = 8
NUM_CLASSES = 4
X_DTYPE = np.float32
Y_DTYPE = np.uint8
MAX_VALUE = 32767

In [6]:
paths, sizes = dataloader.load_dataset_paths(LABELED_PATH, UNLABELED_PATH)

In [7]:
train_brains, val_brains, test_brains, unlabeled_brains = paths

In [8]:
hgg_train_size, lgg_train_size, hgg_val_size, lgg_val_size = sizes 

In [9]:
hgg_train_size, lgg_train_size

(142, 50)

In [7]:
len(train_brains), len(val_brains), len(test_brains), len(unlabeled_brains)

(192, 49, 44, 66)

In [6]:
# all_brains = train_brains + val_brains + test_brains + unlabeled_brains

In [36]:
# dataloader.find_max_per_channel(all_brains)

{'t1': 32767.0, 't1ce': 32767.0, 't2': 32767.0, 'flair': 32767.0}

In [9]:
train_dataset = tf.data.Dataset.from_generator(dataloader.build_data_generator, output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, dataloader.IMAGE_SIZE, dataloader.IMAGE_SIZE, dataloader.CHANNELS), dtype=X_DTYPE),
    tf.TensorSpec(shape=(BATCH_SIZE, dataloader.IMAGE_SIZE, dataloader.IMAGE_SIZE, NUM_CLASSES), dtype=Y_DTYPE)
), args=[train_brains, MAX_VALUE, BATCH_SIZE, BRAIN_SLICES, X_DTYPE, Y_DTYPE, True, hgg_train_size, lgg_train_size])

In [10]:
val_dataset = tf.data.Dataset.from_generator(dataloader.build_data_generator, output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, dataloader.IMAGE_SIZE, dataloader.IMAGE_SIZE, dataloader.CHANNELS), dtype=X_DTYPE),
    tf.TensorSpec(shape=(BATCH_SIZE, dataloader.IMAGE_SIZE, dataloader.IMAGE_SIZE, NUM_CLASSES), dtype=Y_DTYPE)
), args=[val_brains, MAX_VALUE, BATCH_SIZE, BRAIN_SLICES, X_DTYPE, Y_DTYPE, True, hgg_val_size, lgg_val_size])

In [12]:
train_dataset = train_dataset.repeat().prefetch(tf.data.AUTOTUNE)

In [13]:
val_dataset = val_dataset.repeat().prefetch(tf.data.AUTOTUNE)

In [ ]:
# for elem in val_dataset.take(1):
#     plt.imshow(elem[0][0, :, :, 0])
#     elo = elem[0][0, :, :, 0]
#     print(elo.shape)

In [15]:
model = architecture.build_model((dataloader.IMAGE_SIZE, dataloader.IMAGE_SIZE, dataloader.CHANNELS), NUM_CLASSES, 'relu', 'he_normal')

In [30]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 96, 112, 4)]         0         []                            
                                                                                                  
 sequential_15 (Sequential)  (None, 96, 112, 16)          2480      ['input_4[0][0]']             
                                                                                                  
 inception_module_12 (Incep  (None, 48, 56, 16)           824       ['sequential_15[0][0]']       
 tionModule)                                                                                      
                                                                                                  
 sequential_16 (Sequential)  (None, 48, 56, 32)           38176     ['inception_module_12[0]

In [17]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Nadam(), metrics=['accuracy', 
                                                                                               tf.keras.metrics.OneHotMeanIoU(num_classes=NUM_CLASSES),
                                                                                               tf.keras.metrics.OneHotIoU(num_classes=NUM_CLASSES, target_class_ids=[0]),
                                                                                               tf.keras.metrics.OneHotIoU(num_classes=NUM_CLASSES, target_class_ids=[1]),
                                                                                               tf.keras.metrics.OneHotIoU(num_classes=NUM_CLASSES, target_class_ids=[2]),
                                                                                               tf.keras.metrics.OneHotIoU(num_classes=NUM_CLASSES, target_class_ids=[3]),
                                                                                               metrics.weighted_f1,
                                                                                               tf.keras.metrics.Precision(class_id=0),
                                                                                               tf.keras.metrics.Precision(class_id=1),
                                                                                               tf.keras.metrics.Precision(class_id=2),
                                                                                               tf.keras.metrics.Precision(class_id=3),
                                                                                               tf.keras.metrics.Recall(class_id=0),
                                                                                               tf.keras.metrics.Recall(class_id=1),
                                                                                               tf.keras.metrics.Recall(class_id=2),
                                                                                               tf.keras.metrics.Recall(class_id=3)])

In [ ]:
checkpoint_filepath = os.path.join(ROOT_DIR, 'checkpoints', 'model-{epoch:02d}-{val_one_hot_mean_io_u:.3f}')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_one_hot_mean_io_u',
    mode='max',
    save_best_only=True)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, restore_best_weights=True)

csv_logger_callback = tf.keras.callbacks.CSVLogger('brainTumor.log')

In [18]:
history = model.fit(train_dataset, epochs=1, steps_per_epoch=3, validation_data=val_dataset, validation_steps=3, callbacks=[model_checkpoint_callback,
                                                                                                                            early_stop_callback,
                                                                                                                            csv_logger_callback])

3/3 [==============================] - 103s 31s/step - loss: 1.3730 - accuracy: 0.2133 - one_hot_mean_io_u: 0.0545 - one_hot_io_u: 0.2142 - one_hot_io_u_1: 0.0000e+00 - one_hot_io_u_2: 0.0020 - one_hot_io_u_3: 0.0018 - weighted_f1: 0.3980 - precision: 0.0000e+00 - precision_1: 0.0000e+00 - precision_2: 0.0000e+00 - precision_3: 0.0000e+00 - recall: 0.0000e+00 - recall_1: 0.0000e+00 - recall_2: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.2065 - val_accuracy: 0.2397 - val_one_hot_mean_io_u: 0.0607 - val_one_hot_io_u: 0.2408 - val_one_hot_io_u_1: 0.0000e+00 - val_one_hot_io_u_2: 0.0018 - val_one_hot_io_u_3: 0.0000e+00 - val_weighted_f1: 0.4565 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_precision_2: 0.0000e+00 - val_precision_3: 0.0000e+00 - val_recall: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_recall_2: 0.0000e+00 - val_recall_3: 0.0000e+00


In [19]:
history.history

{'loss': [1.3730391263961792],
 'accuracy': [0.21328862011432648],
 'one_hot_mean_io_u': [0.05450255796313286],
 'one_hot_io_u': [0.2141568660736084],
 'one_hot_io_u_1': [0.0],
 'one_hot_io_u_2': [0.002016012789681554],
 'one_hot_io_u_3': [0.0018373513594269753],
 'weighted_f1': [0.3980276584625244],
 'precision': [0.0],
 'precision_1': [0.0],
 'precision_2': [0.0],
 'precision_3': [0.0],
 'recall': [0.0],
 'recall_1': [0.0],
 'recall_2': [0.0],
 'recall_3': [0.0],
 'val_loss': [1.2064648866653442],
 'val_accuracy': [0.23973019421100616],
 'val_one_hot_mean_io_u': [0.06065306067466736],
 'val_one_hot_io_u': [0.2408376783132553],
 'val_one_hot_io_u_1': [0.0],
 'val_one_hot_io_u_2': [0.001774563454091549],
 'val_one_hot_io_u_3': [0.0],
 'val_weighted_f1': [0.4565224349498749],
 'val_precision': [0.0],
 'val_precision_1': [0.0],
 'val_precision_2': [0.0],
 'val_precision_3': [0.0],
 'val_recall': [0.0],
 'val_recall_1': [0.0],
 'val_recall_2': [0.0],
 'val_recall_3': [0.0]}

In [44]:
history = model.fit(train_dataset, epochs=1, steps_per_epoch=len(train_brains) * (dataloader.BRAIN_FRAMES//BATCH_SIZE), validation_data=val_dataset)

Epoch 1/2
768/768 [==============================] - 3951s 5s/step - loss: 0.1080 - accuracy: 0.9780 - val_loss: 0.1126 - val_accuracy: 0.9794
Epoch 2/2
768/768 [==============================] - 169s 220ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.1126 - val_accuracy: 0.9794


In [22]:
model.evaluate()

228/228 [==============================] - 195s 854ms/step - loss: 0.0924 - accuracy: 0.9782


[0.09240186959505081, 0.9781610369682312]